<a href="https://colab.research.google.com/github/SRIKANTH-NAYAK-01/EYE-DISEASE-CLASSIFICATION/blob/main/Eye_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***EYE DISEASE CLASSIFICATION***

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
# importing data from kaggle
!kaggle kernels output srikanthnayak01/eye-disease -p /content

Output file downloaded to /content/resnet50_grayscale.h5
Kernel log downloaded to /content/eye-disease.log 


In [ ]:
!kaggle datasets download -d anirudhcv/labeled-optical-coherence-tomography-oct

100% 6.68G/6.70G [01:25<00:00, 135MB/s]
100% 6.70G/6.70G [01:25<00:00, 84.1MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/labeled-optical-coherence-tomography-oct.zip','r')
zip_ref.extractall('/content')
zip_ref.close()
zip_ref.close()

In [ ]:
#importing necessary libraries
import os
import numpy as np
import cv2
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir = "/content/Dataset - train+val+test/train"
test_dir = "/content/Dataset - train+val+test/test"
val_dir = "/content/Dataset - train+val+test/val"

In [ ]:
#data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=18,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.12,
    zoom_range=0.17,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
     color_mode = 'grayscale'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    color_mode = 'grayscale'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,

    class_mode='categorical',
    color_mode = 'grayscale'

)


Found 76515 images belonging to 4 classes.
Found 21861 images belonging to 4 classes.
Found 10933 images belonging to 4 classes.


In [ ]:
# Load the ResNet-50 model
base_model = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 1))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.6)(x)
predictions = Dense(4, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers[:-20]:  # Freeze fewer layers
    layer.trainable = False



model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 1)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         3200      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

model.fit_generator(
    train_generator,
    validation_data = validation_generator,
    epochs= 10)

model.save('resnet50_grayscale.h5')